In [1]:
import findspark
import os
import gc
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
findspark.init(SPARK_HOME)

import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
import numpy as np
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import size, array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
import geofeather
from pyspark.storagelevel import StorageLevel

from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [2]:
tin_file = input("Here is a programe to extract boundary relations, please input the absolute or relative path to your .ts file:")

# get the directory, basename of the input file
print("\n********************")
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or Tri_data(1) or Tetra_data (2):") or "2"

if directory_type == '0':
    directory = 'hdfs_data'
elif directory_type == '1':
    directory = 'Tri_data'
else:
    directory = 'Tetra_data'
    
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)

print("\n********************")
print("This is a TIN file in \"%s\" format" % tin_extension)

filtra = 'yes'

# allocate the number of executors, the number of cores per executor, and the amount of memory per executor
Num_executor = '64'
Num_core_per_executor = '5'
Memory_executor = '64g'
MemoryOverhead_executor = '8g'

# allocate the number of cores for the driver node
Num_core_per_driver = '5'
Memory_driver = '64g'
MemoryOverhead_driver = '32g'

# the default number of shuffle partitions
Num_shuffle_partitions = input("spark.sql.shuffle.partitions:")


********************
tin_directory:  /home/qiany/yuehui/pyspark/Tetra_mesh/data
tin_basename:  brain.ts
tin_filename:  brain
tin_extension:  .ts

********************
This is a TIN file in ".ts" format


In [3]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

spark_app_name = "TopoRela_Curv_separate_TetraMesh_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '400') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.sql.objectHashAggregate.sortBased.fallbackThreshold", "-1") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config("spark.eventLog.logStageExecutorMetrics", "true") \
.getOrCreate()

spark_app_name: TopoRela_Curv_separate_TetraMesh_brain_06152024_1836


### read input filesread input files

In [4]:
# read_tetra_order() is a function used to read tetrahedron from a csv file
def read_tetra_order(hdfs_tetra_origin):
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a tetrahedra file
    tin_filename: a string denoting the file name of a tetrahedra extension, e.g., 827_monviso
    '''
        
    schema_tetra_origin = StructType([ \
        StructField("tetra_order",IntegerType(),True), \
        StructField("r1",IntegerType(),True), \
        StructField("r2",IntegerType(),True), \
        StructField("r3",IntegerType(),True), \
        StructField("r4",IntegerType(),True), \
        StructField("r1_ele",FloatType(),True), \
        StructField("r2_ele",FloatType(),True), \
        StructField("r3_ele",FloatType(),True), \
        StructField("r4_ele",FloatType(),True) \
      ])

    df_tetra_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tetra_origin)\
          .load(hdfs_tetra_origin)
        
    return df_tetra_origin


# read tetrahedra
hdfs_tetra_origin = directory + "/" + tin_filename + '_filtra_tetra_sort.csv'

df_tetra_order = read_tetra_order(hdfs_tetra_origin)
df_tetra_order.printSchema()

Number of partitions for df_tetra_origin: 200
root
 |-- tetra_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- r4_ele: float (nullable = true)



In [5]:
# read_ver_order() is a function used to read vertices from a csv file
def read_ver_order(hdfs_tin_pts_origin):
    '''
    this function has three input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a TIN file
    tin_filename: a string denoting the file name of a TIN without extension, e.g., Canyon_Lake_Gorge_TX
    '''
    schema_ver_origin = StructType([ \
        StructField("x",FloatType(),True), \
        StructField("y",FloatType(),True), \
        StructField("z",FloatType(),True), \
        StructField("ele",FloatType(),True), \
        StructField("self_index",IntegerType(),True)
      ])
    
    df_ver_order = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_ver_origin)\
          .load(hdfs_tin_pts_origin)
        
    return df_ver_order

hdfs_tin_pts_origin = directory + "/" + tin_filename + '_pts_origin.csv'

df_ver_order = read_ver_order(hdfs_tin_pts_origin)
df_ver_order.printSchema()

root
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- z: float (nullable = true)
 |-- ele: float (nullable = true)
 |-- self_index: integer (nullable = true)



In [6]:
df_ver_order = df_ver_order.withColumn("coor", array('x', 'y', 'z', 'ele')).drop('x', 'y', 'z', 'ele')

df_ver_order.printSchema()

root
 |-- self_index: integer (nullable = true)
 |-- coor: array (nullable = false)
 |    |-- element: float (containsNull = true)



### globally obtain VT relation

In [7]:
# function to get VT directly from DF_T
def get_VT(df_tetra_order):
    df_tetra_order = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False))
    df_VT_init_1 = df_tetra_order.select("r1","tetra")
    df_VT_init_2 = df_tetra_order.select("r2","tetra")
    df_VT_init_3 = df_tetra_order.select("r3","tetra")
    df_VT_init_4 = df_tetra_order.select("r4","tetra")
    
    df_VT_union12 = df_VT_init_1.union(df_VT_init_2)
    df_VT_union123 = df_VT_union12.union(df_VT_init_3)
    df_VT_union1234 = df_VT_union123.union(df_VT_init_4)
    
    df_VT = df_VT_union1234.groupBy('r1').agg(collect_list('tetra').alias('VT'))
    df_VT = df_VT.withColumnRenamed('r1', 'Ver')
    
    return df_VT

In [8]:
df_VT = get_VT(df_tetra_order)

df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [9]:
t0 = time.time()

num_row = df_VT.count()

t1 = time.time()
t_count_VT = t1 - t0
print("Time cost:", t_count_VT)
print("number of rows:", num_row)

Time cost: 24.488054037094116
number of rows: 326683


### globally obtain VF relation

In [10]:
# function to get VF from DF_T
def get_VF_init(df_tetra_order):
    df_VF_1 = df_tetra_order.withColumn("f1", sort_array(F.array("r1", "r2", "r3"), False)).withColumn("f2", sort_array(F.array("r1", "r2", "r4"), False)).withColumn("f3", sort_array(F.array("r1", "r3", "r4"), False)).drop('r2', 'r3', 'r4')
    df_VF_2 = df_tetra_order.withColumn("f1", sort_array(F.array("r2", "r1", "r3"), False)).withColumn("f2", sort_array(F.array("r2", "r1", "r4"), False)).withColumn("f3", sort_array(F.array("r2", "r3", "r4"), False)).drop('r1', 'r3', 'r4')
    df_VF_3 = df_tetra_order.withColumn("f1", sort_array(F.array("r3", "r1", "r2"), False)).withColumn("f2", sort_array(F.array("r3", "r1", "r4"), False)).withColumn("f3", sort_array(F.array("r3", "r2", "r4"), False)).drop('r1', 'r2', 'r4')
    df_VF_4 = df_tetra_order.withColumn("f1", sort_array(F.array("r4", "r1", "r2"), False)).withColumn("f2", sort_array(F.array("r4", "r1", "r3"), False)).withColumn("f3", sort_array(F.array("r4", "r2", "r3"), False)).drop('r1', 'r2', 'r3')
    
    df_VF_union12 = df_VF_1.union(df_VF_2)
    df_VF_union123 = df_VF_union12.union(df_VF_3)
    df_VF_union1234 = df_VF_union123.union(df_VF_4)
    
    df_VF_init = df_VF_union1234.groupBy('r1').agg(collect_set('f1').alias('multi_f1'), collect_set('f2').alias('multi_f2'), collect_set('f3').alias('multi_f3'))
    return df_VF_init

df_VF_init = get_VF_init(df_tetra_order)

# df_VE_init.cache()

df_VF_init.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- multi_f1: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_f2: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_f3: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [11]:
# obtain VF relation
def get_VF(multi_f1, multi_f2, multi_f3):
# get_VF is used to obtain a complete VF relation from the partial VF relations
# multi_f1: partial VF relation
# multi_f2: partial VF relation
# multi_f3: partial VF relation

    faces = set()
    for f in multi_f1:
        faces.add(tuple(f))
        
    for f in multi_f2:
        faces.add(tuple(f))
        
    for f in multi_f3:
        faces.add(tuple(f))
    
    faces_list = sorted(faces) # save more time when using list(faces)
    
    return faces_list

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_VF_udf = udf(get_VF, ArrayType(ArrayType(IntegerType())))

df_VF = df_VF_init.withColumn("VF", get_VF_udf(df_VF_init.multi_f1, df_VF_init.multi_f2, df_VF_init.multi_f3)).drop('multi_f1', 'multi_f2', 'multi_f3')
df_VF = df_VF.withColumnRenamed('r1', 'Ver')

df_VF.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [12]:
t0 = time.time()

num_row = df_VF.count()

t1 = time.time()
t_count_VF = t1 - t0
print("Time cost:", t_count_VF)
print("number of rows:", num_row)

Time cost: 12.739529848098755
number of rows: 326683


### concatenate the relations in one DataFrame

In [13]:
df_Curv_rela_init = df_VT.join(df_VF, df_VT.Ver == df_VF.Ver)
df_Curv_rela_init = df_Curv_rela_init.select(df_VT.Ver, "VT", "VF")

df_Curv_rela_init.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [14]:
def bool_boundary(VT, VF):
    # check if Ver is a boundary vertex or internal vertex
    if 3*len(VT) == 2*len(VF):
        # this is an internal vertex
        return False
    else:
        return True

bool_boundary_udf = udf(bool_boundary, BooleanType())

df_Curv_rela = df_Curv_rela_init.withColumn("BoundaryOrNot", bool_boundary_udf(df_Curv_rela_init.VT, df_Curv_rela_init.VF))
df_Curv_rela.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- BoundaryOrNot: boolean (nullable = true)



In [15]:
t0 = time.time()

num_row = df_Curv_rela.count()

t1 = time.time()
t_count_rela = t1 - t0
print("Time cost:", t_count_rela)
print("number of rows:", num_row)

Time cost: 6.649294137954712
number of rows: 326683


### get coordinates of each vertex

In [16]:
# replace_rer() replaces the three vertices with sorted vertices in df_tetra_origin
def replace_ver(df_ver_order, df_tetra_order):
    '''
    df_ver_order: a DataFrame storing sorted vertices
    df_tetra_order: a DataFrame storing tetrahedra from a TIN
    '''
    df_tetra_order_r1 = df_tetra_order.join(df_ver_order, df_tetra_order.r1 == df_ver_order.self_index, "inner")
    df_tetra_order_r1 = df_tetra_order_r1.select(col("tetra_order"), col("r1"), col("r2"), col("r3"), col("r4"), col("coor").alias("r1_coor"))

    df_tetra_order_r2 = df_tetra_order_r1.join(df_ver_order, df_tetra_order_r1.r2 == df_ver_order.self_index, "inner")
    df_tetra_order_r2 = df_tetra_order_r2.select(col("tetra_order"), col("r1"), col("r2"), col("r3"), col("r4"), col("r1_coor"), col("coor").alias("r2_coor"))

    df_tetra_order_r3 = df_tetra_order_r2.join(df_ver_order, df_tetra_order_r2.r3 == df_ver_order.self_index, "inner")
    df_tetra_order_r3 = df_tetra_order_r3.select(col("tetra_order"), col("r1"), col("r2"), col("r3"), col("r4"), col("r1_coor"), col("r2_coor"), col("coor").alias("r3_coor"))
    
    df_tetra_order_r4 = df_tetra_order_r3.join(df_ver_order, df_tetra_order_r3.r4 == df_ver_order.self_index, "inner")
    df_tetra_order_r4 = df_tetra_order_r4.select(col("tetra_order"), col("r1"), col("r2"), col("r3"), col("r4"), col("r1_coor"), col("r2_coor"), col("r3_coor"), col("coor").alias("r4_coor"))

    df_tetra_order = df_tetra_order_r4.select(col("tetra_order"), col("r1"), col("r2"), col("r3"), col("r4"), col("r1_coor"), col("r2_coor"), col("r3_coor"), col("r4_coor"))

    return df_tetra_order

In [17]:
# replaces extreme vertices with sorted vertices
df_tetra_coor = replace_ver(df_ver_order, df_tetra_order)

df_tetra_coor.printSchema()

root
 |-- tetra_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)
 |-- r1_coor: array (nullable = false)
 |    |-- element: float (containsNull = true)
 |-- r2_coor: array (nullable = false)
 |    |-- element: float (containsNull = true)
 |-- r3_coor: array (nullable = false)
 |    |-- element: float (containsNull = true)
 |-- r4_coor: array (nullable = false)
 |    |-- element: float (containsNull = true)



### compute the trihedal angle of each boundary vertex of a tetrahedron

In [18]:
# we should avoid using NumPy functions in UDF
def compute_trihe_angle(r1, r2, r3, r4, r1_coor, r2_coor, r3_coor, r4_coor):
    r1_coor_cmpt = r1_coor[:3]
    r2_coor_cmpt = r2_coor[:3]
    r3_coor_cmpt = r3_coor[:3]
    r4_coor_cmpt = r3_coor[:3]
    
    def trihe_angle(v1, v2, v3, v4):
        # Compute vectors from v0 to other vertices
        a = [v2[i] - v1[i] for i in range(3)]
        b = [v3[i] - v1[i] for i in range(3)]
        c = [v4[i] - v1[i] for i in range(3)]
        # Compute scalar triple product
        scalar_triple = sum(a[i] * (b[(i+1)%3] * c[(i+2)%3] - b[(i+2)%3] * c[(i+1)%3]) for i in range(3))
        # Compute norms and dot products
        norm_a = math.sqrt(sum(x ** 2 for x in a))
        norm_b = math.sqrt(sum(x ** 2 for x in b))
        norm_c = math.sqrt(sum(x ** 2 for x in c))
        dot_ab = sum(a[i] * b[i] for i in range(3))
        dot_ac = sum(a[i] * c[i] for i in range(3))
        dot_bc = sum(b[i] * c[i] for i in range(3))
        # Compute the solid angle using the formula
        denominator = norm_a * norm_b * norm_c + dot_ab * norm_c + dot_ac * norm_b + dot_bc * norm_a
        solid_angle = 2 * math.atan2(scalar_triple, denominator)
        
        return solid_angle
    
    r1_solid = trihe_angle(r1_coor_cmpt, r2_coor_cmpt, r3_coor_cmpt, r4_coor_cmpt)
    r2_solid = trihe_angle(r2_coor_cmpt, r1_coor_cmpt, r3_coor_cmpt, r4_coor_cmpt)
    r3_solid = trihe_angle(r3_coor_cmpt, r1_coor_cmpt, r2_coor_cmpt, r4_coor_cmpt)    
    r4_solid = trihe_angle(r4_coor_cmpt, r1_coor_cmpt, r2_coor_cmpt, r3_coor_cmpt)
    r_solid = [r1_solid, r2_solid, r3_solid, r4_solid]
    
    r_ary = [r1, r2, r3, r4]
    r_ary_sort = sorted(r_ary, reverse=True) 
    
    solid_ary_sort = [] 
    final_v1_index = r_ary.index(r_ary_sort[0])
    solid_ary_sort.append(r_solid[final_v1_index])   
    final_v2_index = r_ary.index(r_ary_sort[1])
    solid_ary_sort.append(r_solid[final_v2_index])  
    final_v3_index = r_ary.index(r_ary_sort[2])
    solid_ary_sort.append(r_solid[final_v3_index])  
    final_v4_index = r_ary.index(r_ary_sort[3])
    solid_ary_sort.append(r_solid[final_v4_index])
    
    return r_ary_sort, solid_ary_sort

# StructType       
compute_trihe_angle_schema = StructType([
    StructField('ary_sort', ArrayType(IntegerType()),True), 
    StructField('solid_ary_sort', ArrayType(FloatType()),True)
])

compute_trihe_angle_udf = udf(compute_trihe_angle, compute_trihe_angle_schema)

df_tetra_solid_init = df_tetra_coor.withColumn("T_cur", compute_trihe_angle_udf(df_tetra_coor.r1, df_tetra_coor.r2, df_tetra_coor.r3, df_tetra_coor.r4, df_tetra_coor.r1_coor, df_tetra_coor.r2_coor, df_tetra_coor.r3_coor, df_tetra_coor.r4_coor))
# df_tetra_solid = df_tetra_solid_init.select("tetra_order", "r1", "r2", "r3", "r4", col("T_cur.ary_sort").alias("tetra"), col("T_cur.solid_ary_sort").alias("tetra_solid"))
df_tetra_solid = df_tetra_solid_init.select(col("T_cur.ary_sort").alias("tetra"), col("T_cur.solid_ary_sort").alias("tetra_solid"))

df_tetra_solid.printSchema()

root
 |-- tetra: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- tetra_solid: array (nullable = true)
 |    |-- element: float (containsNull = true)



In [19]:
df_Curv_rela_init = df_Curv_rela.select("Ver", "BoundaryOrNot", explode("VT").alias("single_tetra"))
df_Curv_init_exp = df_Curv_rela_init.join(df_tetra_solid, df_Curv_rela_init.single_tetra == df_tetra_solid.tetra).drop('tetra_order', 'tetra')

df_Curv_init_exp.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- BoundaryOrNot: boolean (nullable = true)
 |-- single_tetra: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- tetra_solid: array (nullable = true)
 |    |-- element: float (containsNull = true)



In [20]:
def get_v_cur(Ver, single_tetra, tetra_solid):
    index_v = single_tetra.index(Ver)
    cur_v = tetra_solid[index_v]
    
    return cur_v

get_v_cur_udf = udf(get_v_cur, FloatType())
df_Curv_init = df_Curv_init_exp.withColumn("v_solid", get_v_cur_udf(df_Curv_init_exp.Ver, df_Curv_init_exp.single_tetra, df_Curv_init_exp.tetra_solid)).drop('tetra_solid')
df_Curv_init.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- BoundaryOrNot: boolean (nullable = true)
 |-- single_tetra: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- v_solid: float (nullable = true)



In [21]:
df_Curv_gp = df_Curv_init.groupby("Ver", "BoundaryOrNot").agg(collect_list("v_solid").alias("solid_values"))
df_Curv_gp.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- BoundaryOrNot: boolean (nullable = true)
 |-- solid_values: array (nullable = true)
 |    |-- element: float (containsNull = true)



In [22]:
def compute_cur(BoundaryOrNot, solid_values):
    # discrete distortion for internal vertices
    if BoundaryOrNot == False:
        distortion = 4*3.14 - sum(solid_values)
    else: # boundary vertices
        distortion = sum(solid_values)
    return distortion

compute_cur_udf = udf(compute_cur, FloatType())

df_Curv = df_Curv_gp.withColumn("discrete_distortion", compute_cur_udf(df_Curv_gp.BoundaryOrNot, df_Curv_gp.solid_values))
df_Curv.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- BoundaryOrNot: boolean (nullable = true)
 |-- solid_values: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- discrete_distortion: float (nullable = true)



In [23]:
t0 = time.time()

num_row = df_Curv.count()

t1 = time.time()
t_count_Curv = t1 - t0
print("Time cost:", t_count_Curv)
print("number of rows:", num_row)

Time cost: 35.425727128982544
number of rows: 326683


In [24]:
print("****************************")
print("Time cost for preparing VT:", t_count_VT)
print("Time cost for preparing VF:", t_count_VF)
print("Time cost for concatenate the relations:", t_count_rela)
print("Total time cost for preparing the relations:", t_count_VT + t_count_VF + t_count_rela)

print("****************************")
print("Time cost for calculating the discrete distortion:", t_count_Curv)

print("****************************")
print("Total time cost:", t_count_VT + t_count_VF + t_count_rela + t_count_Curv)

****************************
Time cost for preparing VT: 24.488054037094116
Time cost for preparing VF: 12.739529848098755
Time cost for concatenate the relations: 6.649294137954712
Total time cost for preparing the relations: 43.87687802314758
****************************
Time cost for calculating the discrete distortion: 35.425727128982544
****************************
Total time cost: 79.30260515213013
